# Dataset Generation

In [1]:
import os
os.chdir(f'{os.getcwd()}/../../')
WORKING_DIR = './evaluation/trigger'

In [2]:
import numpy as np
import pandas as pd

In [3]:
shc_mse = np.load(f'{WORKING_DIR}/data/shc_rmse.npy')
shr_psnr = np.load(f'{WORKING_DIR}/data/shr_psnr.npy')
shr_ssim = np.load(f'{WORKING_DIR}/data/shr_ssim.npy')

In [4]:
for m in range(3):
    metric = [shc_mse, shr_psnr, shr_ssim][m]
    metric_name = ['shc_mse', 'psnr', 'ssim'][m]
    
    for i in range(25):
        index = np.arange(metric.shape[0])
        zeros = np.zeros_like(index)
        ones = np.ones_like(index)

        l = np.quantile(metric, (i + 1) / 100)
        l_mask = metric <= l
        l_data = metric[l_mask]

        r = np.quantile(metric, (100 - (i + 1)) / 100)
        r_mask = metric >= r
        r_data = metric[r_mask]
        
        decision =  np.concatenate((zeros[l_mask], ones[r_mask]), axis=0)
        if m > 0:
            decision = 1 - decision

        df = pd.DataFrame.from_dict({
            'idx': np.concatenate((index[l_mask], index[r_mask]), axis=0),
            'diff': np.concatenate((l_data, r_data), axis=0),
            'metric_name': metric_name,
            'decision': decision
        })

        df.to_csv(f'{WORKING_DIR}/dataset/{metric_name}_{(i + 1)}_{(100 - (i + 1))}.csv')